# Summary

This notebook is set up to preprocess the data. While preprocessing, the data will be visualized for further understanding. The goal is to clean and understand the data to ultimately set it up for feature selection and training under models.

The preprocessing steps contain:
- log transformation and standardization of numerical data, and 
- encoding categorical data.

## 1. Import Libraries & Initialize Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from scipy import stats
from scipy.stats import mstats
from sklearn.base import TransformerMixin
import numpy.ma as ma

# changed display options to accomodate for long list in output
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 2000)

In [ ]:
# can load ~1/6 of dataset so will split into 3 chunks
n_chunks=6

# total num of rows in OG file
n_rows=sum(1 for row in open('train.csv')) -1 # subtract header row

chunk_size=n_rows // n_chunks

chunk = next(pd.read_csv('train.csv', chunksize=chunk_size, low_memory=False))
print(f'chunk shape - before : {chunk.shape}')

In [ ]:
# parameters

# drop col with NA > than threshold (50%)
NA_thresh = 0.5
# outlier threshold
out_thresh=3

# define features
cat_col=chunk.select_dtypes(include='object').columns
time_col=['VAR_0073','VAR_0075','VAR_0156','VAR_0157',
          'VAR_0158','VAR_0159','VAR_0166','VAR_0167','VAR_0168','VAR_0169',
           'VAR_0176','VAR_0177','VAR_0178','VAR_0179','VAR_0204','VAR_0217','VAR_0314','VAR_0531']
num_col=chunk.select_dtypes(include=np.number).columns.difference(time_col)

In [ ]:
processor_ck=Prep(chunk,time_col)
prepped_ck=processor_ck.process_df()

print(f'Chunk shape - after: {prepped_ck.shape}')

In [ ]:
# global variables 
cat_col=prepped_ck.select_dtypes(include='object').columns
num_col=prepped_ck.select_dtypes(include=np.number).columns.difference(time_col)

In [ ]:
processor_ck=NA(prepped_ck,NA_thresh,num_col,cat_col)

# initial NA count
print(f'Number of NA - before: {np.any(prepped_ck.isnull())}')

# apply NA function
imputed_ck = processor_ck.process_na()

# post NA count
print(f'Number of NA - after: {np.any(imputed_ck.isnull())}')
print(f'Chunk shape - after: {imputed_ck.shape}')

## 2. Preprocessing

### 2.1 transforming numerical data

Will apply:
1. Log Transformation - makes initial data distribution more normal and handles skewed data
2. Standardization - ensures all features are on the same scale

Before applying log transformation, negative values must be dealt with to ensure code is not taking the log of zero or negative values. Will take measures to shift the data to positive before applying log transformation.

In [ ]:
class ShiftPositive(TransformerMixin):
    def __init__(self,target_col=None):
        self.target_col=target_col
    def fit(self,X,y=None):
        return self
    
    # test if float column contains only integer values
    def test_type(self,colvector):
        return colvector[colvector.notnull()].apply(
            lambda x: x.is_integer()).sum() == len(colvector[colvector.notnull()])
        
    def transform(self, X, y=None, int_amount=1, deci_amount=0.1):
        if self.target_col:
            X = X.drop(columns=[self.target_col])
        
        # separate columns into integer vs float-valued
        int_cols = X.dtypes[X.dtypes == np.dtype('int64')].index.tolist()
        float_cols = X.dtypes[X.dtypes == np.dtype('float64')].index.tolist()

        int_with_nans_bool = X[float_cols].apply(self.test_type)
        int_with_nans = int_with_nans_bool[int_with_nans_bool].index.tolist()
        int_cols.extend(int_with_nans)
        float_cols = list(set(float_cols).difference(set(int_with_nans)))

        # preserve categorical columns
        cat_cols = X.select_dtypes(include='object').columns.tolist()

        # shift integer columns and float columns based on parameters
        new_int_df = X[int_cols].apply(lambda x: x - x.min() + int_amount if x.min() <= 0 else x)
        new_float_df = X[float_cols].apply(lambda x: x - x.min() + deci_amount if x.min() <= 0 else x)

        # merge categorical columns back with numerical
        transformed_df = pd.concat([new_int_df, new_float_df, X[cat_cols]], axis=1)
        
        if self.target_col and self.target_col in y:
            transformed_df[self.target_col] = y
            
        return transformed_df, int_cols, float_cols, cat_cols

In [ ]:
shift_pos=ShiftPositive()

In [ ]:
shifted_ck, int_cols, float_cols, cat_cols=shift_pos.fit_transform(imputed_ck)

# print('integer cols:', int_cols)
# print('float cols:', float_cols)
# shifted_ck['target']

In [ ]:
class LogTransform(TransformerMixin):
    def __init__(self, target_col=None):
        self.target_col = target_col
    def fit(self,x,y=None):
        return self
    
    # test if feature is normally distributed
    def dist_test(self,colvec,test='normal'):
        if colvec.dtype.kind not in 'biufc':  # numeric column check
            return np.nan
        if test == 'normal':
            return stats.mstats.normaltest(colvec[colvec.notnull()])[0]
        elif test == 'skew':
            return stats.mstats.skewtest(colvec[colvec.notnull()])[0]
        elif test == 'kurtosis':
            return stats.mstats.kurtosistest(colvec[colvec.notnull()])[0]
        else:
            print('unknown test type')
            return
        
    # applies log transformation to cols identified using dist_test
    def transform(self,X,y=None,thresh=5000):
        if self.target_col in X.columns:
            X = X.drop(columns=[self.target_col])
        
        # preserve categorical columns
        cat_cols = X.select_dtypes(include='object').columns.tolist()
        
        # Apply normal test and determine columns to transform
        test_results = X.apply(self.dist_test)
        to_transform_cols = test_results[test_results > thresh].index.tolist()

        # Apply log transform to identified columns
        transformed_cols = X[to_transform_cols].apply(lambda x: np.log(x + 1) if test_results[x.name] > thresh else x)  # Shift positive value before transforming
        
        # Combine with non-transformed columns
        unchanged_cols = list(set(X.columns) - set(to_transform_cols))
        transformed_df = pd.concat([X[unchanged_cols], transformed_cols], axis=1)

        return transformed_df

In [ ]:
log_transformer=LogTransform()

In [ ]:
transformed_ck=log_transformer.fit_transform(shifted_ck)
# print(transformed_ck.head(2))
# print('transformed_ck["target"]')

In [ ]:
def standardize(df,target_col=None):
    if target_col and target_col in df.columns:
        target=df[target_col]
        df = df.drop(columns=[target_col])
    else:
        target=None
    
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    scaler=StandardScaler()
    standardize=scaler.fit_transform(df[num_cols])
    standardized_df = pd.DataFrame(standardize, columns=num_cols, index=df.index)
    
    # combine with non-numerical columns
    non_numeric_cols = df.select_dtypes(exclude=['float64', 'int64']).columns
    if not non_numeric_cols.empty:
        standardized_df = pd.concat([standardized_df, df[non_numeric_cols]], axis=1)
    
    if target is not None:
        standardized_df[target_col] = target
    
    return standardized_df, scaler

In [ ]:
standardized_ck, scaler=standardize(transformed_ck,target_col='target')
# standardized_ck['target']

### 2.2 outliers

In [ ]:
class Outliers:
    def __init__(self,df,out_thresh):
        self.df=df
        self.out_thresh=out_thresh
        self.outlier_counts={}
        
    def calc_bounds(self,col):
        Q1 = self.df[col].quantile(0.25)
        Q3 = self.df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        return lower_bound, upper_bound
    
    def count_outliers(self, col, lower_bound, upper_bound):
        return ((self.df[col] < lower_bound) | (self.df[col] > upper_bound)).sum()
    
    def remove_outlier(self):
        outlier_df = self.df.copy()
        
        for col in outlier_df.select_dtypes(include=[np.number]).columns:
            lower_bound, upper_bound = self.calc_bounds(col)
            outlier_counts = self.count_outliers(col, lower_bound, upper_bound)
            self.outlier_counts[col]=outlier_counts
            outlier_df = outlier_df[(outlier_df[col] >= lower_bound) | (outlier_df[col] <= upper_bound)]

        return outlier_df
    
    def display_outlier(self):
        for col, count in self.outlier_counts.items():
            print(f'Number of outliers in {col}: {count}')

In [ ]:
processor_ck=Outliers(standardized_ck,out_thresh=3)
cleaned_ck = processor_ck.remove_outlier()
# processor_ck.display_outlier()

### 2.3 Encoding Categorical Variables

Encoding the categories are straightforward since most of them  have under 10 unique values with no specific levels of ordering; so one-hot encoding will be the standard.

Exception: There are a few categories with 50+ unique values (states & cities). Will use grouping & one-hot encode - reduce number of categories by mapping states into regions/larger geographic areas.

#### 2.3.1 Group & one-hot

In [ ]:
region_mapping = {
    'Northeast': ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA'],
    'Midwest': ['IN', 'IL', 'MI', 'OH', 'WI', 'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD'],
    'South': ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'DC', 'WV', 'AL', 'KY', 'MS', 'TN', 'AR', 'LA', 'OK', 'TX'],
    'West': ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY', 'AK', 'CA', 'HI', 'OR', 'WA']
}

flattened_region_mapping = {}
for region, states in region_mapping.items():
    for state in states:
        flattened_region_mapping[state] = region
        
# flattened_region_mapping

In [ ]:
class RegionProcessor:
    def __init__(self,df,region_mapping):
        self.df=df
        self.region_mapping=region_mapping
        
    def create_col(self):
        self.df['REGION_0237']=self.df['VAR_0237'].map(self.region_mapping)
        self.df['REGION_0274']=self.df['VAR_0274'].map(self.region_mapping)
        
    def drop_col(self):
        self.df.drop(columns=['VAR_0237','VAR_0274','VAR_0342','VAR_0200'], inplace=True)
        
    def process_df(self):
        self.create_col()
        self.drop_col()
        return self.df

In [ ]:
processor=RegionProcessor(cleaned_ck,flattened_region_mapping)
grouped_ck=processor.process_df()

In [ ]:
class OneHotEncoder:
    def __init__(self,df,columns):
        self.df=df
        self.columns=columns
        
    def one_hot(self):
        self.df=pd.get_dummies(self.df,columns=self.columns)
        
    def process_df(self):
        self.one_hot()
        return self.df

In [ ]:
# define cat col
categorical_col=grouped_ck.select_dtypes(include=['object']).columns

# instantiate class and process
encoder=OneHotEncoder(grouped_ck,categorical_col)
encoded_ck=encoder.process_df()

# print(encoded_ck.head(2))

## 3. Save Data

### 3.1 Train

In [ ]:
# save train chunk
encoded_ck.to_csv('train_ck.csv', index=False)
print('train chunk has been saved')

### 3.2 Test

In [ ]:
# split into test chunk

# can load ~1/6 of dataset so will split into 3 chunks
n_chunks=6

# total num of rows in OG file
n_rows=sum(1 for row in open('test.csv')) -1 # subtract header row

chunk_size=n_rows // n_chunks

test_chunk = next(pd.read_csv('test.csv', chunksize=chunk_size, low_memory=False))
print(f'chunk shape - before : {test_chunk.shape}')

In [ ]:
# parameters for test chunk

# drop col with NA > than threshold (50%)
NA_thresh = 0.5
# outlier threshold
out_thresh=3

# define features
cat_col=test_chunk.select_dtypes(include='object').columns
time_col=['VAR_0073','VAR_0075','VAR_0156','VAR_0157',
          'VAR_0158','VAR_0159','VAR_0166','VAR_0167','VAR_0168','VAR_0169',
           'VAR_0176','VAR_0177','VAR_0178','VAR_0179','VAR_0204','VAR_0217','VAR_0314','VAR_0531']
num_col=test_chunk.select_dtypes(include=np.number).columns.difference(time_col)

In [ ]:
# prep test

processor_ck=Prep(test_chunk,time_col)
prepped_test=processor_ck.process_df()

print(f'Chunk shape - after: {prepped_test.shape}')

In [ ]:
# NA test

processor_ck=NA(prepped_test,NA_thresh,num_col,cat_col)

# initial NA count
print(f'Number of NA - before: {np.any(prepped_test.isnull())}')

# apply NA function
imputed_test = processor_ck.process_na()

# post NA count
print(f'Number of NA - after: {np.any(imputed_test.isnull())}')
print(f'Chunk shape - after: {imputed_test.shape}')

In [ ]:
# shifting positive test chunk
shifted_test, int_cols, float_cols, cat_cols=shift_pos.fit_transform(imputed_test)

# log transofroming test chunk
transformed_test=log_transformer.fit_transform(shifted_test)

In [ ]:
# standardized test chunk
standardized_test, scaler=standardize(transformed_test)

In [ ]:
processor_test=Outliers(standardized_test,out_thresh=3)
cleaned_test = processor_test.remove_outlier()

In [ ]:
processor=RegionProcessor(cleaned_test,flattened_region_mapping)
grouped_test=processor.process_df()

In [ ]:
# define cat col
categorical_col=grouped_test.select_dtypes(include=['object']).columns

# instantiate class and process
encoder=OneHotEncoder(grouped_test,categorical_col)
encoded_test=encoder.process_df()

In [ ]:
# save test chunk
encoded_test.to_csv('test_ck.csv', index=False)
print('test chunk has been saved')